##Input Csv to Pandas

In [0]:
import pandas as pd
import numpy as np

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
dataTestFileID = "1KNaMPpNwYPTWteYDcBH3eaAH40Hd0hbD"
dataTrainFileID = "1-0v6W8gQiFfMgdAOs-cP1gh6-jBHnMhz"
dataTrain2FileID = "1CrH_a9P5GNcu-GEntz7DL-K3XXueVkOv"

In [0]:
from io import StringIO
testFile = drive.CreateFile({'id': dataTestFileID})
trainFile = drive.CreateFile({'id': dataTrainFileID})
# train2File = drive.CreateFile({'id': dataTrain2FileID})
testBuffer = StringIO(testFile.GetContentString())
trainBuffer = StringIO(trainFile.GetContentString())
# train2Buffer = StringIO(train2File.GetContentString())

In [0]:
dataTest = pd.read_csv(testBuffer, index_col=0)
dataTest = dataTest.dropna()
dataTrain = pd.read_csv(trainBuffer, index_col=0)
dataTrain = dataTrain.dropna()
# dataTrain2 = pd.read_csv(train2Buffer, index_col=0)
# dataTrain2.columns = ["Text","Verdict"]
# dataTrain2 = dataTrain2.dropna()

In [18]:


# #if using the big dataTrain
# dataTrain = dataTrain2
# dataTest = dataTest.drop((dataTest.Verdict==2).index)
dataTest.info()
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
Text       0 non-null object
Verdict    0 non-null int64
dtypes: int64(1), object(1)
memory usage: 0.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
Text       0 non-null object
Verdict    0 non-null int64
dtypes: int64(1), object(1)
memory usage: 0.0+ bytes


##Training Doc2Vec Model

In [7]:
!pip install -U gensim
import multiprocessing
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from gensim.models.doc2vec import TaggedDocument
cores = multiprocessing.cpu_count()

assert gensim.models.doc2vec.FAST_VERSION > -1

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


###Prepare tagged data train

In [0]:
taggedDataTrain = []
for row in dataTrain.itertuples(index=True,name="Pandas"):
  text = row.Text  
  words = text.split()    
  verdict = getattr(row,"Verdict")
  tags = [str(verdict), str(row.Index)]
  taggedDataTrain.append(TaggedDocument(words,tags))  

###Initialize model

In [0]:
model = Doc2Vec(dm=1, alpha = 0.02, min_alpha = 0.0025, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=1, sample=0, epochs=20, workers=cores)
model.build_vocab(taggedDataTrain)

###Train the model

In [14]:
for epoch in range(10):
  print("Now training epoch",epoch+1)
  model.train(taggedDataTrain, total_examples = model.corpus_count, epochs = model.iter )
  model.alpha-= 0.0002
  model.min_alpha=model.alpha

Now training epoch 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Now training epoch 2
Now training epoch 3
Now training epoch 4
Now training epoch 5
Now training epoch 6
Now training epoch 7
Now training epoch 8
Now training epoch 9
Now training epoch 10


##Testing

In [0]:
testWords = [ text.split() for text in dataTest["Text"] ]
testVerdict = [ verdict for verdict in dataTest["Verdict"] ]

In [16]:
testPredict = []
for words in testWords:
  v1 = model.infer_vector(words)
  sim = model.docvecs.most_similar([v1])   
  
  #voting system
  
  simTrainIdx = int(sim[0][0])
  
  
  predict = int(dataTrain["Verdict"][simTrainIdx])    
    
  testPredict.append(predict)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


###Check Fitness

In [17]:
score = 0.
for idx in range(len(testPredict)):
  prediction = testPredict[idx]
  verdict = testVerdict[idx]
  if (prediction == verdict):
    score+=1
    
fitness = score/len(testWords)
print(fitness)

0.3694925112694489
